#Libraries Import

In [5]:
!pip install xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.2 MB/s eta 0:00:00


In [40]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

#Importing Our Lists of Stock

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

# Making Our API Call

In [8]:
!pip install pandas==1.3.3 pandas-datareader==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.10.0
    Uninstalling pandas-datareader-0.10.0:
      Successfully uninstalled pandas-datareader-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.20.0 requires pandas>=1.5.0, but you have pandas 1.3.3 which is incompatible.
bigframes 1.27.0 requires pandas>=1.5.3, but you have pandas 1.3.3 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.3.3 which is incompatible.
geopandas 1.0.1 requires pandas>=1.4.0,

In [3]:
import pandas_datareader as pdr
import datetime

In [65]:
symbol = 'AAPL'

In [11]:
import yfinance as yf
import pandas as pd


# Fetch the ticker object
ticker = yf.Ticker(symbol)

# Extract fundamental and historical data
info = ticker.info
history = ticker.history(period="max")

# function to calculate percentage changes
def calculate_change(history, days):
    if len(history) < days:
        return None
    start_price = history['Close'].iloc[-days]
    end_price = history['Close'].iloc[-1]
    return (end_price - start_price) / start_price

# data dictionary
data = {
    "week52change": (info.get("52WeekChange") or None),
    "week52high": info.get("fiftyTwoWeekHigh", None),
    "week52low": info.get("fiftyTwoWeekLow", None),
    "marketcap": info.get("marketCap", None),
    "employees": info.get("fullTimeEmployees", None),
    "day200MovingAvg": info.get("twoHundredDayAverage", None),
    "day50MovingAvg": info.get("fiftyDayAverage", None),
    "float": info.get("floatShares", None),
    "avg10Volume": info.get("averageDailyVolume10Day", None),
    "avg30Volume": info.get("averageVolume", None),
    "ttmEPS": info.get("trailingEps", None),
    "ttmDividendRate": info.get("trailingAnnualDividendRate", None),
    "companyName": info.get("shortName", None),
    "sharesOutstanding": info.get("sharesOutstanding", None),
    "maxChangePercent": None,  # Calculation requires different data source which is not in yfinance
    "year5ChangePercent": calculate_change(history, 5 * 252),  # Approx. 252 trading days/year
    "year2ChangePercent": calculate_change(history, 2 * 252),
    "year1ChangePercent": calculate_change(history, 252),
    "ytdChangePercent": calculate_change(history.loc[f"{pd.Timestamp.now().year}-01-01":], len(history)),
    "month6ChangePercent": calculate_change(history, 126),  # Approx. 21 trading days/month * 6
    "month3ChangePercent": calculate_change(history, 63),
    "month1ChangePercent": calculate_change(history, 21),
    "day30ChangePercent": calculate_change(history, 30),
    "day5ChangePercent": calculate_change(history, 5),
    "nextDividendDate": info.get("dividendDate", None),
    "dividendYield": info.get("dividendYield", None),
    "nextEarningsDate": info.get("nextEarningsDate", None),
    "exDividendDate": info.get("exDividendDate", None),
    "peRatio": info.get("forwardPE", None),
    "beta": info.get("beta", None),
}

for key, value in data.items():
    print(f"{key}: {value}")


week52change: 0.2616992
week52high: 244.105
week52low: 164.08
marketcap: 3673290440704
employees: 164000
day200MovingAvg: 206.4068
day50MovingAvg: 229.4358
float: 15091184209
avg10Volume: 43672210
avg30Volume: 48371926
ttmEPS: 6.09
ttmDividendRate: 0.98
companyName: Apple Inc.
sharesOutstanding: 15115799552
maxChangePercent: None
year5ChangePercent: 2.8644746732626336
year2ChangePercent: 0.6613591035073918
year1ChangePercent: 0.2625400977439985
ytdChangePercent: None
month6ChangePercent: 0.25235992568520454
month3ChangePercent: 0.10169976582338551
month1ChangePercent: 0.08873288590145219
day30ChangePercent: 0.054244053225681425
day5ChangePercent: 0.034393232379680096
nextDividendDate: None
dividendYield: 0.0041
nextEarningsDate: None
exDividendDate: 1731024000
peRatio: 29.25188
beta: 1.24


# Parsing Our API Call

In [12]:
data['year1ChangePercent']

0.2625400977439985

#Executing A Batch API Call & Building Our DataFrame

In [13]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [14]:
# for symbol_string in symbol_strings:
#   print(symbol_string)

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [15]:
import yfinance as yf
import pandas as pd

# split the list into chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

stocks = pd.read_csv('sp_500_stocks.csv')

# Split the stock tickers into groups of 100
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = [','.join(group) for group in symbol_groups]

# Initialize an empty DataFrame
final_dataframe = pd.DataFrame(columns=my_columns)

# Loop through each batch of symbols
for symbols in symbol_groups:
    for symbol in symbols:
        try:
            # Fetch data for each symbol
            ticker = yf.Ticker(symbol)
            info = ticker.info
            history = ticker.history(period="1y")  # Get 1-year historical data

            # Calculate 1-year percentage change
            if not history.empty:
                year_change_percent = ((history["Close"].iloc[-1] - history["Close"].iloc[0]) /
                                       history["Close"].iloc[0]) * 100
            else:
                year_change_percent = None  # Handle cases with no data

            # Append the data to the DataFrame
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        info.get("currentPrice", None),  # Latest price
                        year_change_percent,  # 1-year price return
                        "N/A"
                    ],
                    index=my_columns
                ),
                ignore_index=True
            )

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

# Save the final DataFrame to a CSV file
final_dataframe.to_csv("sp500_data.csv", index=False)

# Display the final DataFrame
print(final_dataframe)


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=UxTnW4FUwMo
ERROR:yfinance:$ABC: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=UxTnW4FUwMo
ERROR:yfinance:$ABMD: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$ALXN: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$ANTM: possibly delisted; no price data fo

    Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  140.20              10.37744                     N/A
1      AAL   14.88             13.328256                     N/A
2      AAP   44.01            -15.701309                     N/A
3     AAPL  243.01              26.25401                     N/A
4     ABBV  176.46             26.520369                     N/A
..     ...     ...                   ...                     ...
500    YUM  138.16             13.289504                     N/A
501    ZBH  110.62             -3.692555                     N/A
502   ZBRA  411.94             75.772317                     N/A
503   ZION   59.17             64.047029                     N/A
504    ZTS  175.32             -1.443694                     N/A

[505 rows x 4 columns]


# Removing LOW-MOMENTUM STOCKS

In [16]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,140.20,10.37744,N/A
1,AAL,14.88,13.328256,N/A
2,AAP,44.01,-15.701309,N/A
3,AAPL,243.01,26.25401,N/A
4,ABBV,176.46,26.520369,N/A
...,...,...,...,...
500,YUM,138.16,13.289504,N/A
501,ZBH,110.62,-3.692555,N/A
502,ZBRA,411.94,75.772317,N/A
503,ZION,59.17,64.047029,N/A


In [17]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,145.14,211.753536,N/A
1,UAL,99.25,150.694619,N/A
2,HWM,120.09,130.84882,N/A
3,HBI,8.58,127.586206,N/A
4,RCL,249.74,119.024975,N/A
5,NRG,100.93,114.020756,N/A
6,TPR,63.83,103.870712,N/A
7,SYF,66.82,101.048118,N/A
8,NFLX,911.06,100.16698,N/A
9,ANET,105.43,95.503216,N/A


#Calculating the Number of Shares to Buy

In [18]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_input()

In [20]:
portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [30]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number Of Shares To Buy,Number of Shares to Buy
0,NVDA,145.14,211.753536,1350.0,1350
1,UAL,99.25,150.694619,1975.0,1975
2,HWM,120.09,130.84882,1632.0,1632
3,HBI,8.58,127.586206,22852.0,22852
4,RCL,249.74,119.024975,785.0,785
5,NRG,100.93,114.020756,1942.0,1942
6,TPR,63.83,103.870712,3071.0,3071
7,SYF,66.82,101.048118,2934.0,2934
8,NFLX,911.06,100.16698,215.0,215
9,ANET,105.43,95.503216,1859.0,1859


# Building a Better(and More Realistic) Momentum Strategy

In [52]:
from datetime import timedelta

# Define the columns for the dataframe
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

# Iterate over all stocks in the CSV
for symbol in stocks['Ticker']:
    try:
        # Fetch historical data for the ticker
        ticker = yf.Ticker(symbol)
        history = ticker.history(period="1y")  #1 year of historical data

        # Initializing variables for returns
        year_change_percent = None
        month_6_change_percent = None
        month_3_change_percent = None
        month_1_change_percent = None

        if not history.empty:
            # Ensure the index is a DatetimeIndex
            history.reset_index(inplace=True)
            history.set_index("Date", inplace=True)

            # Calc. the most recent price and percentage changes
            end_date = history.index[-1]
            start_date_6_months = end_date - timedelta(days=6*30)
            start_date_3_months = end_date - timedelta(days=3*30)
            start_date_1_month = end_date - timedelta(days=1*30)

            # One-Year Return
            year_change_percent = ((history["Close"].iloc[-1] - history["Close"].iloc[0])
                                   / history["Close"].iloc[0]) * 100

            # Six-Month Return
            six_month_data = history[history.index >= start_date_6_months]
            if not six_month_data.empty:
                month_6_change_percent = (
                    (six_month_data["Close"].iloc[-1] - six_month_data["Close"].iloc[0]) /
                    six_month_data["Close"].iloc[0]
                ) * 100

            # Three-Month Return
            three_month_data = history[history.index >= start_date_3_months]
            if not three_month_data.empty:
                month_3_change_percent = (
                    (three_month_data["Close"].iloc[-1] - three_month_data["Close"].iloc[0]) /
                    three_month_data["Close"].iloc[0]
                ) * 100

            # One-Month Return
            one_month_data = history[history.index >= start_date_1_month]
            if not one_month_data.empty:
                month_1_change_percent = (
                    (one_month_data["Close"].iloc[-1] - one_month_data["Close"].iloc[0]) /
                    one_month_data["Close"].iloc[0]
                ) * 100

        # Appending data to the dataframe
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    ticker.info.get("currentPrice", None),
                    "N/A",  # Placeholder for "Number of Shares to Buy"
                    year_change_percent,
                    "N/A",  # Placeholder for "One-Year Return Percentile"
                    month_6_change_percent,
                    "N/A",  # Placeholder for "Six-Month Return Percentile"
                    month_3_change_percent,
                    "N/A",  # Placeholder for "Three-Month Return Percentile"
                    month_1_change_percent,
                    "N/A",  # Placeholder for "One-Month Return Percentile"
                    "N/A"   # Placeholder for "HQM SCORE"
                ],
                index=hqm_columns,
            ),
            ignore_index=True,
        )

    except Exception as e:
        print(f"Failed to process {symbol}: {e}")

# Display the resulting dataframe
hqm_dataframe


ERROR:yfinance:$ABC: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=UxTnW4FUwMo
ERROR:yfinance:$ABMD: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=UxTnW4FUwMo
ERROR:yfinance:$ALXN: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$ANTM: possibly delisted; no price data fo

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.20,N/A,10.37744,N/A,5.753433,N/A,2.349261,N/A,0.393844,N/A,N/A
1,AAL,14.88,N/A,13.328256,N/A,29.50392,N/A,37.777776,N/A,14.549656,N/A,N/A
2,AAP,44.01,N/A,-15.701309,N/A,-29.599098,N/A,11.43401,N/A,17.516682,N/A,N/A
3,AAPL,243.01,N/A,26.25401,N/A,26.117935,N/A,10.169977,N/A,9.579461,N/A,N/A
4,ABBV,176.46,N/A,26.520369,N/A,5.305101,N/A,-8.030435,N/A,-11.976852,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.16,N/A,11.078957,N/A,0.253977,N/A,3.204608,N/A,4.067497,N/A,N/A
501,ZBH,110.62,N/A,-4.489723,N/A,-2.002121,N/A,5.543369,N/A,2.038556,N/A,N/A
502,ZBRA,411.94,N/A,75.772317,N/A,36.133508,N/A,28.422239,N/A,9.262112,N/A,N/A
503,ZION,59.17,N/A,58.250867,N/A,43.338178,N/A,31.401288,N/A,15.65676,N/A,N/A


# Calculating Momentum Percentiles


*   One-Year Price Return
*   Six-Month Price Return
*   Three-Month Price Return
*   One-Month Price Return



In [53]:
from scipy.stats import percentileofscore

# Time periods for return calculations
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

# Calculate percentiles for each time period
for time_period in time_periods:
    change_col = f'{time_period} Price Return'
    percentile_col = f'{time_period} Return Percentile'

    # Ensure the percentile column exists
    hqm_dataframe[percentile_col] = 0.0

    for row in hqm_dataframe.index:
        value = hqm_dataframe.loc[row, change_col]

        # Calculate the percentile for the current row's value
        if pd.notnull(value):  # Avoid calculating percentiles for NaN values
            hqm_dataframe.loc[row, percentile_col] = percentileofscore(
                hqm_dataframe[change_col].dropna(),
                value
            ) / 100  # Convert to decimal percentage


In [54]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.20,N/A,10.37744,0.302832,5.753433,0.357298,2.349261,0.359477,0.393844,0.283224,N/A
1,AAL,14.88,N/A,13.328256,0.342048,29.50392,0.788671,37.777776,0.965142,14.549656,0.915033,N/A
2,AAP,44.01,N/A,-15.701309,0.080610,-29.599098,0.026144,11.43401,0.601307,17.516682,0.958606,N/A
3,AAPL,243.01,N/A,26.25401,0.559913,26.117935,0.753813,10.169977,0.557734,9.579461,0.747277,N/A
4,ABBV,176.46,N/A,26.520369,0.570806,5.305101,0.344227,-8.030435,0.139434,-11.976852,0.017429,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.16,N/A,11.078957,0.318083,0.253977,0.246187,3.204608,0.374728,4.067497,0.459695,N/A
501,ZBH,110.62,N/A,-4.489723,0.163399,-2.002121,0.215686,5.543369,0.435730,2.038556,0.359477,N/A
502,ZBRA,411.94,N/A,75.772317,0.947712,36.133508,0.884532,28.422239,0.888889,9.262112,0.736383,N/A
503,ZION,59.17,N/A,58.250867,0.873638,43.338178,0.943355,31.401288,0.919390,15.65676,0.938998,N/A


# Calculating HQM Score

In [55]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.20,N/A,10.37744,0.302832,5.753433,0.357298,2.349261,0.359477,0.393844,0.283224,0.325708
1,AAL,14.88,N/A,13.328256,0.342048,29.50392,0.788671,37.777776,0.965142,14.549656,0.915033,0.752723
2,AAP,44.01,N/A,-15.701309,0.080610,-29.599098,0.026144,11.43401,0.601307,17.516682,0.958606,0.416667
3,AAPL,243.01,N/A,26.25401,0.559913,26.117935,0.753813,10.169977,0.557734,9.579461,0.747277,0.654684
4,ABBV,176.46,N/A,26.520369,0.570806,5.305101,0.344227,-8.030435,0.139434,-11.976852,0.017429,0.267974
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.16,N/A,11.078957,0.318083,0.253977,0.246187,3.204608,0.374728,4.067497,0.459695,0.349673
501,ZBH,110.62,N/A,-4.489723,0.163399,-2.002121,0.215686,5.543369,0.435730,2.038556,0.359477,0.293573
502,ZBRA,411.94,N/A,75.772317,0.947712,36.133508,0.884532,28.422239,0.888889,9.262112,0.736383,0.864379
503,ZION,59.17,N/A,58.250867,0.873638,43.338178,0.943355,31.401288,0.919390,15.65676,0.938998,0.918845


#Selecting the 50 Best Momentum Stocks

In [56]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,454,UAL,99.25,N/A,150.694619,0.997821,87.158217,1.000000,115.713983,1.000000,28.478964,0.997821,0.998911
1,390,RCL,249.74,N/A,118.514309,0.991285,60.707861,0.989107,59.527308,0.993464,22.187977,0.986928,0.990196
2,441,TPR,63.83,N/A,96.884656,0.984749,44.444453,0.949891,58.151637,0.991285,31.445634,1.000000,0.981481
3,215,HBI,8.58,N/A,127.586206,0.993464,67.578127,0.995643,35.759489,0.949891,20.505619,0.978214,0.979303
4,426,SYF,66.82,N/A,96.645087,0.982571,50.699142,0.965142,40.94073,0.971678,21.358514,0.982571,0.97549
5,84,CCL,26.78,N/A,62.303034,0.899782,62.008467,0.991285,70.899812,0.997821,22.227297,0.989107,0.969499
6,195,FTNT,98.19,N/A,86.957353,0.976035,63.404896,0.993464,30.641295,0.910675,24.543388,0.995643,0.968954
7,332,NFLX,911.06,N/A,100.16698,0.986928,41.359193,0.938998,36.84305,0.958606,20.588739,0.980392,0.966231
8,459,UNM,75.67,N/A,77.963305,0.956427,47.418655,0.958606,39.767268,0.967320,18.08676,0.965142,0.961874
9,293,LYV,136.73,N/A,67.952327,0.932462,52.345399,0.973856,47.37012,0.982571,16.653866,0.949891,0.959695


In [58]:
portfolio_input()

Enter the value of your portfolio:10000000


In [59]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe['Price'][i])
hqm_dataframe

/usr/local/lib/python3.10/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,454,UAL,99.25,2015,150.694619,0.997821,87.158217,1.000000,115.713983,1.000000,28.478964,0.997821,0.998911
1,390,RCL,249.74,800,118.514309,0.991285,60.707861,0.989107,59.527308,0.993464,22.187977,0.986928,0.990196
2,441,TPR,63.83,3133,96.884656,0.984749,44.444453,0.949891,58.151637,0.991285,31.445634,1.000000,0.981481
3,215,HBI,8.58,23310,127.586206,0.993464,67.578127,0.995643,35.759489,0.949891,20.505619,0.978214,0.979303
4,426,SYF,66.82,2993,96.645087,0.982571,50.699142,0.965142,40.94073,0.971678,21.358514,0.982571,0.97549
5,84,CCL,26.78,7468,62.303034,0.899782,62.008467,0.991285,70.899812,0.997821,22.227297,0.989107,0.969499
6,195,FTNT,98.19,2036,86.957353,0.976035,63.404896,0.993464,30.641295,0.910675,24.543388,0.995643,0.968954
7,332,NFLX,911.06,219,100.16698,0.986928,41.359193,0.938998,36.84305,0.958606,20.588739,0.980392,0.966231
8,459,UNM,75.67,2643,77.963305,0.956427,47.418655,0.958606,39.767268,0.967320,18.08676,0.965142,0.961874
9,293,LYV,136.73,1462,67.952327,0.932462,52.345399,0.973856,47.37012,0.982571,16.653866,0.949891,0.959695


# Formatting Our Excel Output

In [60]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)


##Creating the Formats We'll Need For Our .xlsx File


* String format for tickers


*  $XX.XX format for stock prices

* $XX,XXX format for market capitalization

*   Integer format for the number of shares to purchase





In [61]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [63]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
  writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
  writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)


In [64]:
##Saving the Excel
writer.save()